The packages used were `tidyverse`, `tidyxl`, `readxl`, `dplyr`, and `janitor`.

`tidyverse` was used to wrangle the data into tidy dataframes. \
`readxl` was used to read the .xlsx format data, and `tidyxl` was used to extract the data from within specific sheets of the file. \
`dplyr` was used to \
`janitor` was used to

In [18]:
# import required libraries
library(tidyverse)
library(tidyxl)
library(readxl)
library(dplyr)
library(janitor)

The data was obtained from StatsNZ, and can be found by following the link "https://www.stats.govt.nz/information-releases/statistical-area-1-dataset-for-2018-census-updated-march-2020". From here, the "Regional files (Microsoft Excel Open XML)" for the Canterbury region is to be downloaded. This consists of a zipped file containing six _.xlsx_ files. Four of these files are used in the subsequent wrangling: *2018-SA1-dataset-households-CanterburyRegion*, *2018-SA1-dataset-individual-part-1-CanterburyRegion_updated_28-7-20*, *2018-SA1-dataset-individual-part-2-CanterburyRegion*, and *2018-SA1-dataset-individual-part-3a-CanterburyRegion*. These files have been as part of this project for simplicity, to *canterbury_households.xlsx*, *canterbury_population.xlsx*, *canterbury_income.xlsx*, and *canterbury_employment.xslx*, respectively.

The file containing Canterbury income data was read using the `read_excel()` function. The required sheet was labelled "SA2", which consists of a number of columns referencing various suburb-level features, however we are focusing specifically on those related to income brackets.

When the data was read, it structured the tibble column names by adding the string '...x' to the end, where x refers to the column index value. This string was removed using the `sub()` function. The "Area_Description" column name was also changed to "Suburb" for simplicity.

In [19]:
# load income data
income_data <- read_excel("Region_data_income.xlsx", 
                                 sheet = "SA2",
                                 col_names = TRUE,
                                 skip = 9)
income_data <- income_data %>% filter_all(any_vars(!is.na(.)))
income_data <- income_data[,-c(1)]

colnames(income_data) <-  sub("\\...*", "", colnames(income_data))
names(income_data)[names(income_data) == "Area_Description"] <- "Suburb"

head(income_data)

New names:
* `No religion` -> `No religion...3`
* Buddhism -> Buddhism...4
* Christian -> Christian...5
* Hinduism -> Hinduism...6
* Islam -> Islam...7
* ...



Suburb,No religion,Buddhism,Christian,Hinduism,Islam,Judaism,"Māori religions, beliefs and philosophies",Spiritualism and New Age religions,"Other religions, beliefs and philosophies",...,Bicycle,Walk or jog,School bus,Public bus,Train,Ferry,Other,Total stated,Not elsewhere included,Total
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Kaikoura Ranges,435,6,789,6,0,0,6,3,3,...,0,0,21,3,0,0,0,78,0,78
Kaikoura,702,9,1179,0,3,3,21,12,3,...,21,42,81,0,0,0,0,324,0,324
Hanmer Range,63,0,114,0,0,0,0,0,0,...,C,C,C,C,C,C,C,C,C,6
Amuri,657,3,1098,3,0,3,6,3,3,...,9,63,216,6,3,0,0,453,0,453
Hanmer Springs,258,3,303,3,3,3,0,6,0,...,18,15,3,0,0,0,0,99,0,99
Upper Hurunui,390,6,834,3,0,0,0,6,0,...,12,18,144,0,0,0,0,252,0,252


The required columns relating to income were extracted, as well as the suburb column. Several unneccessary columns were dropped

In [20]:
# extracting 2018 individual income information and convert to dataframe
canterbury_income <- income_data[c(1, 264:274)]

canterbury_income = canterbury_income[-11]
canterbury_income = canterbury_income[-10]
canterbury_income = canterbury_income[-9]

head(canterbury_income)

Suburb,"$5,000 or less","$5,001 – $10,000","$10,001 – $20,000","$20,001 – $30,000","$30,001 – $50,000","$50,001 – $70,000","$70,001 or more",Median personal income($)
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Kaikoura Ranges,105,57,219,201,354,219,213,35600
Kaikoura,108,69,357,414,492,279,189,30200
Hanmer Range,21,6,30,21,63,33,33,37100
Amuri,174,57,267,207,402,351,258,38600
Hanmer Springs,51,39,114,132,255,102,114,35300
Upper Hurunui,114,57,258,189,237,171,159,28300


In [21]:
# extracting 2018 individual income information, convert to dataframe, rename columns to dataframe-appropriate values
canterbury_religion <- income_data[c(1:9)]

head(canterbury_religion)

Suburb,No religion,Buddhism,Christian,Hinduism,Islam,Judaism,"Māori religions, beliefs and philosophies",Spiritualism and New Age religions
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Kaikoura Ranges,435,6,789,6,0,0,6,3
Kaikoura,702,9,1179,0,3,3,21,12
Hanmer Range,63,0,114,0,0,0,0,0
Amuri,657,3,1098,3,0,3,6,3
Hanmer Springs,258,3,303,3,3,3,0,6
Upper Hurunui,390,6,834,3,0,0,0,6


In order to get more specific regional information, the same file was read again, this time specifying the "Geographic key" sheet. This sheet provides detailed Canterbury region data, of which columns the *Statistical area 2 description*, *Territorial authority description*, and *Regional council description* were required. These were renamed to "Suburb", "Territory" and "Region", respectively, for simplicity.

In [22]:
# load region data
region <- read_excel("Region_data_income.xlsx",
                     sheet = "Geographic key",
                     col_names = TRUE,
                     skip = 2)
head(region)

Statistical area 1 code (2018 areas),Statistical area 2 code (2018 areas),Statistical area 2 description,Territorial authority code (2018 areas),Territorial authority description,Ward code (2018 areas),Ward description,District health board code (2015 areas),District health board description,Regional council code (2018 areas),Regional council description
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
7023547,308800,Kaikoura Ranges,054,Kaikoura District,05499,Area Outside Ward,18,Canterbury,13,Canterbury Region
7023548,308800,Kaikoura Ranges,054,Kaikoura District,05499,Area Outside Ward,18,Canterbury,13,Canterbury Region
7023549,308800,Kaikoura Ranges,054,Kaikoura District,05499,Area Outside Ward,18,Canterbury,13,Canterbury Region
7023550,308800,Kaikoura Ranges,054,Kaikoura District,05499,Area Outside Ward,18,Canterbury,13,Canterbury Region
7023551,308800,Kaikoura Ranges,054,Kaikoura District,05499,Area Outside Ward,18,Canterbury,13,Canterbury Region
7023552,308800,Kaikoura Ranges,054,Kaikoura District,05499,Area Outside Ward,18,Canterbury,13,Canterbury Region


In [23]:
# filter to get just suburbs/areas in Canterbury and convert to dataframe
region <- region[c(3, 5, 11)] %>% distinct()
names(region)[names(region) == "Territorial authority description"] <- "Territory"
names(region)[names(region) == "Statistical area 2 description"] <- "Suburb"
names(region)[names(region) == "Regional council description"] <- "Region"

region <- region[region$Region == "Canterbury Region",]
region = data.frame(region)
region

Suburb,Territory,Region
<chr>,<chr>,<chr>
Kaikoura Ranges,Kaikoura District,Canterbury Region
Kaikoura,Kaikoura District,Canterbury Region
Hanmer Range,Hurunui District,Canterbury Region
Amuri,Hurunui District,Canterbury Region
Hanmer Springs,Hurunui District,Canterbury Region
Parnassus,Hurunui District,Canterbury Region
Upper Hurunui,Hurunui District,Canterbury Region
Omihi,Hurunui District,Canterbury Region
Ashley Forest,Hurunui District,Canterbury Region


In [71]:
# rename columns to dataframe appropriate values
names(canterbury_income)[2] <- "5k_or_less"
names(canterbury_income)[3] <- "5k_to_10k"
names(canterbury_income)[4] <- "10k_to_20k"
names(canterbury_income)[5] <- "20k_to_30k"
names(canterbury_income)[6] <- "30k_to_50k"
names(canterbury_income)[7] <- "50k_to_70k"
names(canterbury_income)[8] <- "70k_or_more"
names(canterbury_income)[9] <- "Median_personal_income"

canterbury_income[, 2:9] = sapply(canterbury_income[, 2:9], as.integer)
canterbury_income = data.frame(canterbury_income)

canterbury_income = na.omit(canterbury_income)

head(canterbury_income)

,Suburb,X5k_or_less,X5k_to_10k,X10k_to_20k,X20k_to_30k,X30k_to_50k,X50k_to_70k,X70k_or_more,Median_personal_income
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Kaikoura Ranges,105,57,219,201,354,219,213,35600
2,Kaikoura,108,69,357,414,492,279,189,30200
3,Hanmer Range,21,6,30,21,63,33,33,37100
4,Amuri,174,57,267,207,402,351,258,38600
5,Hanmer Springs,51,39,114,132,255,102,114,35300
6,Upper Hurunui,114,57,258,189,237,171,159,28300


In [72]:
write.csv(canterbury_income, "canterbury_income.csv")

The same process used above was followed to extract the following data tables: 

In [73]:
# -------- PEOPLE DATASET --------

# load people data
people_data <- read_excel("Region_data_people.xlsx", 
                                 sheet = "SA2",
                                 col_names = TRUE,
                                 skip = 9)
people_data <- people_data %>% filter_all(any_vars(!is.na(.)))
people_data <- people_data[,-c(1)]

colnames(people_data) <-  sub("\\...*", "", colnames(people_data))
names(people_data)[names(people_data) == "Area_Description"] <- "Suburb"

head(people_data)

New names:
* `2006 Census` -> `2006 Census...3`
* `2013 Census` -> `2013 Census...4`
* `2018 Census` -> `2018 Census...5`
* `2006 Census` -> `2006 Census...6`
* `2013 Census` -> `2013 Census...7`
* ...



Suburb,2006 Census,2013 Census,2018 Census,2006 Census,2013 Census,2018 Census,2018 Census individual form,Individuals on the household listing only(16),Admin enumeration(16),...,Don't know,Total stated,Not elsewhere included,Total,Māori descent,No Māori descent,Don't know,Total stated,Not elsewhere included,Total
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Kaikoura Ranges,1419,1545,1689,1551,1650,1737,1383,48,261,...,39,1371,171,1545,336,1314,42,1689,0,1689
Kaikoura,2202,2010,2223,3333,2814,2898,1785,57,381,...,36,1803,204,2010,471,1701,51,2223,0,2223
Hanmer Range,216,249,261,405,546,336,222,6,33,...,9,228,21,249,15,237,9,261,0,261
Amuri,2001,2097,2223,2019,2088,2202,1953,63,207,...,39,1908,189,2097,285,1878,60,2223,0,2223
Hanmer Springs,729,840,960,1617,1476,1488,813,48,102,...,9,735,102,840,87,858,15,960,0,960
Upper Hurunui,1365,1425,1473,1338,1422,1434,1284,36,153,...,21,1302,126,1425,138,1293,45,1473,0,1473


In [76]:
# Extracting relevant data from the people dataset - sex, age - and rename columns to dataframe-appropriate values
canterbury_sex <- people_data[c(1, 18:19)]

names(canterbury_sex)[2] <- "Male_count"
names(canterbury_sex)[3] <- "Female_count"

canterbury_sex[, 2:3] = sapply(canterbury_sex[, 2:3], as.integer)
canterbury_sex = data.frame(na.omit(canterbury_sex))

head(canterbury_sex)

Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"
Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"


,Suburb,Male_count,Female_count
,<chr>,<int>,<int>
1,Kaikoura Ranges,894,795
2,Kaikoura,1107,1116
3,Hanmer Range,141,123
4,Amuri,1173,1047
5,Hanmer Springs,480,480
6,Upper Hurunui,747,729


,Suburb,X0.4.years....Male,X0.4.years....Female,X5.9.years....Male,X5.9.years....Female,X10.14.years....Male,X10.14.years....Female,X15.19.years....Male,X15.19.years....Female,X20.24.years....Male,...,X80.84.years....Male,X80.84.years....Female,X85.years.and.over....Male,X85.years.and.over....Female,Total.Male,Total.Female,Total,Median.age..Male,Median.age..Female,Median.age..Total
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,Kaikoura Ranges,39,45,45,48,51,57,48,48,24,...,12,9,3,9,717,702,1419,41.8,40.4,41.2
2,Kaikoura,51,57,63,72,69,69,57,60,63,...,27,33,15,18,1071,1131,2202,41.4,41.2,41.2
3,Hanmer Range,6,3,6,9,3,9,6,6,3,...,0,0,0,0,111,105,216,45.6,41.7,44.4
4,Amuri,90,78,78,72,90,84,69,54,51,...,6,9,9,6,1050,954,2001,36.6,37.2,37.0
5,Hanmer Springs,33,18,15,30,18,12,15,9,21,...,6,3,3,3,363,369,729,40.5,39.8,40.0
6,Upper Hurunui,66,42,54,51,60,51,51,36,21,...,15,9,9,9,702,663,1365,41.2,40.8,41.0


In [77]:
write.csv(canterbury_sex, "canterbury_sex.csv")

In [78]:
# -------- EMPLOYMENT DATASET --------

# load employment data
employment_data <- read_excel("Region_data_work.xlsx", 
                                 sheet = "SA2",
                                 col_names = TRUE,
                                 skip = 9)
employment_data <- employment_data %>% filter_all(any_vars(!is.na(.)))
employment_data <- employment_data[,-c(1)]

colnames(employment_data) <-  sub("\\...*", "", colnames(employment_data))
names(employment_data)[names(employment_data) == "Area_Description"] <- "Suburb"

head(employment_data)

New names:
* `Employed Full time` -> `Employed Full time...3`
* `Employed Part time` -> `Employed Part time...4`
* Unemployed -> Unemployed...5
* `Not in the Labour Force` -> `Not in the Labour Force...6`
* `Total stated` -> `Total stated...7`
* ...



Suburb,Employed Full time,Employed Part time,Unemployed,Not in the Labour Force,Total stated,Work and Labour Force Status Unidentifiable,Total,Employed Full time,Employed Part time,...,Professional Scientific and Technical Services,Administrative and Support Services,Public Administration and Safety,Education and Training,Health Care and Social Assistance,Arts and Recreation Services,Other Services,Total stated,Not Elsewhere Included,Total
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Kaikoura Ranges,615,198,15,249,1080,54,1131,633,246,...,27,12,12,18,9,9,9,567,0,567
Kaikoura,900,324,27,507,1758,57,1818,741,294,...,51,63,66,75,63,24,42,1239,0,1239
Hanmer Range,102,36,0,27,165,15,183,114,45,...,3,3,0,6,0,12,0,126,0,126
Amuri,840,249,24,327,1437,72,1509,903,276,...,27,15,9,75,21,3,21,1176,0,1176
Hanmer Springs,321,87,12,126,549,54,603,393,108,...,24,15,9,18,18,63,12,558,0,558
Upper Hurunui,561,195,15,258,1032,12,1044,582,207,...,15,6,6,54,24,3,18,555,0,555


In [63]:
# extracting 2018 individual income information, convert to dataframe, rename columns, change column data types
canterbury_employment <- data.frame(employment_data[c(1, 16:19)])

names(canterbury_employment)[2] <- "Employed_full_time"
names(canterbury_employment)[3] <- "Employed_part_time"
names(canterbury_employment)[4] <- "Unemployed"
names(canterbury_employment)[5] <- "Not_in_labour_force"

canterbury_employment[, 2:5] = sapply(canterbury_employment[, 2:5], as.integer)

head(employment_2018)

Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"
Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"
Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"
Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"


,Suburb,Employed_full_time,Employed_part_time,Unemployed,Not_in_labour_force
,<chr>,<int>,<int>,<int>,<int>
1,Kaikoura Ranges,834,246,15,282
2,Kaikoura,969,315,30,591
3,Hanmer Range,123,39,0,48
4,Amuri,1047,297,30,339
5,Hanmer Springs,498,147,6,162
6,Upper Hurunui,594,207,24,357


In [79]:
# Read crime data to extract columns for suburb and territory. Wrangle into a cleaner form
crime_region = read_csv("VICTIM_TIMEPLACE_Migrated_Data_new.csv", col_names = TRUE)

crime_region <- data.frame(crime_region[,-c(1:4, 6:11, 13:17)])

names(crime_region)[names(crime_region) == "Area.Unit"] <- "Suburb"
names(crime_region)[names(crime_region) == "Territorial.Authority"] <- "Territory"

crime_region$Suburb <- substr(as.character(crime_region$Suburb),
                             start = 1,
                             stop = nchar(as.character(crime_region$Suburb))-1)
crime_region$Territory <- substr(as.character(crime_region$Territory),
                             start = 1,
                             stop = nchar(as.character(crime_region$Territory))-1)

head(crime_region)


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  `Month Year` = col_character(),
  `ANZSOC Division` = col_character(),
  `ANZSOC Group` = col_character(),
  `ANZSOC Subdivision` = col_character(),
  `Area Unit` = col_character(),
  `Location Type` = col_character(),
  `Locn Type Division` = col_character(),
  Meshblock = col_double(),
  `Occurrence Day Of Week` = col_character(),
  `Occurrence Hour Of Day` = col_double(),
  `Table 1` = col_character(),
  `Territorial Authority` = col_character(),
  Weapon = col_character(),
  `Year Month` = col_character(),
  `Year Month (copy 2)` = col_character(),
  `Number of Records` = col_double(),
  Victimisations = col_double()
)




,Suburb,Territory
,<chr>,<chr>
1,Ellerslie South,Auckland
2,Epsom Central,Auckland
3,Glendowie,Auckland
4,Lynnmall,Auckland
5,Mt Victoria,Auckland
6,Onehunga South East,Auckland


In [80]:
# Filtering data to retrieve all distinct suburbs in Canterbury
crime_region = distinct(crime_region, Suburb, .keep_all = TRUE)
crime_region <- filter(crime_region, Territory %in% c("Waimakariri District", "Hurunui District", "Waitaki District",
                                                              "Christchurch City", "Ashburton District", "Timaru District",
                                                               "Waimate District", "Selwyn District", "Mackenzie District",
                                                               "Kaikoura District"))
head(crime_region)

,Suburb,Territory
,<chr>,<chr>
1,Sockburn,Christchurch City
2,Hillmorton,Christchurch City
3,Rutland,Christchurch City
4,Northcote,Christchurch City
5,Avon Loop,Christchurch City
6,Mona Vale,Christchurch City


In [70]:
# Anti-join income data with crime dataset suburbs to find which suburbs are not of conventional naming standard
region_SA2 = anti_join(canterbury_income, crime_region, by = "Suburb")
region_SA2

Suburb,X5k_or_less,X5k_to_10k,X10k_to_20k,X20k_to_30k,X30k_to_50k,X50k_to_70k,X70k_or_more,Median_personal_income
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Kaikoura Ranges,105,57,219,201,354,219,213,35600
Kaikoura,108,69,357,414,492,279,189,30200
Hanmer Range,21,6,30,21,63,33,33,37100
Upper Hurunui,114,57,258,189,237,171,159,28300
Ashley Forest,72,18,75,60,117,84,105,36400
Omihi,117,42,189,147,219,156,141,31000
Balcairn,201,84,357,303,387,303,294,31100
Starvation Hill-Cust,210,60,273,228,327,258,402,36700
Ashley-Sefton,210,60,261,222,324,276,351,36700


In [90]:
# Remove the directions from suburb names, as these are usually the cause of mismatched rows
directions = c("North", "East", "South", "Wesr")

region_SA2 = region_SA2 %>%
    mutate(Suburb = str_remove_all(Suburb, " North| East| South| West")) %>%
    na.omit()

head(region_SA2)

,Suburb,X5k_or_less,X5k_to_10k,X10k_to_20k,X20k_to_30k,X30k_to_50k,X50k_to_70k,X70k_or_more,Median_personal_income
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Kaikoura Ranges,105,57,219,201,354,219,213,35600
2,Kaikoura,108,69,357,414,492,279,189,30200
3,Hanmer Range,21,6,30,21,63,33,33,37100
4,Upper Hurunui,114,57,258,189,237,171,159,28300
5,Ashley Forest,72,18,75,60,117,84,105,36400
6,Omihi,117,42,189,147,219,156,141,31000


In [93]:
# Group by suburb and sum all columns (except median_personal_income, which is taken as a mean) to combine necessary suburb data
agg_region_SA2 = region_SA2 %>% 
    group_by(Suburb) %>% 
    summarise(X5k_or_less = sum(X5k_or_less),
              X5k_to_10k = sum(X5k_to_10k),
              X10k_to_20k = sum(X10k_to_20k),
              X20k_to_30k = sum(X20k_to_30k),
              X30k_to_50k = sum(X30k_to_50k),
              X50k_to_70k = sum(X50k_to_70k),
              X70k_or_more = sum(X70k_or_more),
              Median_person_income = mean(Median_personal_income, na.rm = TRUE))
agg_region_SA2 = data.frame(agg_region_SA2)
head(agg_region_SA2)

,Suburb,X5k_or_less,X5k_to_10k,X10k_to_20k,X20k_to_30k,X30k_to_50k,X50k_to_70k,X70k_or_more,Median_person_income
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,Addington,534,210,840,684,1227,759,399,30950
2,Allenton,339,129,600,645,801,648,594,34300
3,Arundel,114,48,126,138,246,201,210,38800
4,Ashburton,57,27,159,204,177,111,99,27200
5,Ashburton Central,12,9,21,21,27,18,21,31100
6,Ashburton Forks,195,60,216,177,414,372,285,41800
